# **Pipeline**

In [1]:
# importe as principais bibliotecas
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

## **TOC:**
Na aula de hoje, vamos explorar os seguintes tópicos em Python:

- 1) [Introdução ao Problema](#intro)
    - 1.1) [Stratify](#stratify)
    - 1.2) [Transformando os dados](#transform)

- 2) [Pipeline](#pipe)
- 3) [Grid search](#grid)
    - 3.1) [Cross validation](#crossvalidation)

---

## 1) **Introdução ao Problema** <a class="anchor" id="intro"></a>

In [2]:
bunch = load_breast_cancer(as_frame=True)
X, y = bunch["data"], bunch["target"]

### 1.1) **Stratify** <a class="anchor" id="stratify"></a>


Nesse dataset o target não esta balanceado.

In [3]:
y.value_counts()

1    357
0    212
Name: target, dtype: int64

Quando separamos o dataset queremos que a distribuição de probabilidades da variável target no treino e no test sejam as mesmas. 
Para fazer isso utilizamos o parâmetro ```stratify```.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=42,
                                                    stratify=y)

In [5]:
y_train.value_counts(normalize=True)

1    0.626374
0    0.373626
Name: target, dtype: float64

In [6]:
y_test.value_counts(normalize=True)

1    0.631579
0    0.368421
Name: target, dtype: float64

Em outras palavras, a quantidade de cada desfecho da variável ```y``` é igual no treino e no test.

### 1.2) **Transformando os dados** <a class="anchor" id="transform"></a>

O primeiro processo que pode ser feito para tunning do KNN é a **transformação das features**. 

Devido ao KNN utilizar **distâncias** como critério para classificação, ter as variaveis transformadas garante que sejam eliminados viéses relacionados à escala dos dados.

Para **normalizar os dados**, o sklearn nos apresenta a ferramenta [standard scaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

In [7]:
# redefinir pra poder alterar novamente
from sklearn.neighbors import KNeighborsClassifier

# normalizando as features com o standardscaler
from sklearn.preprocessing import StandardScaler 

# instanciando a classe
scaler = StandardScaler()

# transformando as features de treino com o método fit_transform()
X_train = scaler.fit_transform(X_train)

# fitando o modelo novamente
estimador = KNeighborsClassifier()
modelo = estimador.fit(X_train, y_train)

# transformando também as features de teste com o mesmo scaler! PASSO IMPORTANTE!
# importante: use apenas o método transform()
X_test = scaler.transform(X_test)

# faça as predições com as features de teste transformadas
y_pred = modelo.predict(X_test)

# avalie o modelo
from sklearn.metrics import classification_report, confusion_matrix

print("Matriz de confusão do modelo nos dados de teste:\n")
print(confusion_matrix(y_test, y_pred))

print("\nMatriz de confusão do modelo nos dados de teste:\n")
print(classification_report(y_test, y_pred))

Matriz de confusão do modelo nos dados de teste:

[[39  3]
 [ 2 70]]

Matriz de confusão do modelo nos dados de teste:

              precision    recall  f1-score   support

           0       0.95      0.93      0.94        42
           1       0.96      0.97      0.97        72

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



Outra opção é transformar os dados com o [min max scaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)

In [8]:
# redefinir pra poder alterar novamente
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=42,
                                                    stratify=y)

# normalizando as features com o standardscaler

from sklearn.preprocessing import MinMaxScaler

# instanciando a classe
scaler = MinMaxScaler()

# transformando as features de treino com o método fit_transform()
X_train = scaler.fit_transform(X_train)

# fitando o modelo novamente
estimador = KNeighborsClassifier()
modelo = estimador.fit(X_train, y_train)

# transformando também as features de teste com o mesmo scaler! PASSO IMPORTANTE!
# importante: use apenas o método transform()
X_test = scaler.transform(X_test)

# faça as predições com as features de teste transformadas
y_pred = modelo.predict(X_test)

# avalie o modelo
from sklearn.metrics import classification_report, confusion_matrix

print("Matriz de confusão do modelo nos dados de teste:\n")
print(confusion_matrix(y_test, y_pred))

print("\nMatriz de confusão do modelo nos dados de teste:\n")
print(classification_report(y_test, y_pred))

Matriz de confusão do modelo nos dados de teste:

[[40  2]
 [ 0 72]]

Matriz de confusão do modelo nos dados de teste:

              precision    recall  f1-score   support

           0       1.00      0.95      0.98        42
           1       0.97      1.00      0.99        72

    accuracy                           0.98       114
   macro avg       0.99      0.98      0.98       114
weighted avg       0.98      0.98      0.98       114



<font color = "orange"><b>Importante:</b></font> treine os scalers **apenas nos dados de treino** para evitar que informação dos dados de teste sejam passadas para o scaler! (Mais informações [aqui](https://datascience.stackexchange.com/questions/38395/standardscaler-before-and-after-splitting-data))

Observe como a performance do modelo mudou!

Mas será que é possível melhorar ainda mais? Vamos agora aprender como é possível testar diferentes parâmetros de uma única vez!

____

______

## 2) **Pipeline** <a class="anchor" id="intro"></a>

O [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) é uma estrutura que o sklearn proporciona a fim de garantir que possamos em uma única estrutura passar **o estimador e o transformador**. Para maiores informações, [clique aqui](https://scikit-learn.org/stable/modules/compose.html#pipeline). 

Vamos ver a seguir como esta estrutura funciona:

In [9]:
# redefinir pra poder alterar novamente
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=42,
                                                    stratify=y)

# importando a classe de pipeline
from sklearn.pipeline import Pipeline

# o que queremos utilizar para construir o modelo
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# instanciando a classe
pipe = Pipeline([('scaler', StandardScaler()),
                 ('knn', KNeighborsClassifier())])

# fitamos o pipeline: ambas as operações são aplicadas!
# note que o objeto que é fitado é o pipeline!
pipe.fit(X_train, y_train)

# fazendo predições 
# o pipeline garante que o scaler não usará dados de teste!!
y_pred = pipe.predict(X_test)

# testando o modelo
# exatamente o mesmo que acima!
from sklearn.metrics import classification_report, confusion_matrix

print("Matriz de confusão do modelo nos dados de teste:\n")
print(confusion_matrix(y_test, y_pred))

print("\nMatriz de confusão do modelo nos dados de teste:\n")
print(classification_report(y_test, y_pred))

Matriz de confusão do modelo nos dados de teste:

[[39  3]
 [ 2 70]]

Matriz de confusão do modelo nos dados de teste:

              precision    recall  f1-score   support

           0       0.95      0.93      0.94        42
           1       0.96      0.97      0.97        72

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



______

## 3) **Gridsearch** <a class="anchor" id="grid"></a>


O [Gridsearch](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) é uma estrutura que o sklearn proporciona para que seja feita **a busca exaustiva de hiperparâmetros de um modelo**.

Na prática, o que o gridsearch faz é **treinar diversos modelos com diferentes combinações de hiperparâmetros**, de modo a manter o melhor deles como o modelo final, tudo automaticamente! Tudo o que precisamos fazer é indicar quais os hiperparâmetros que queremos procurar: a **grade**!

É muito comum também passarmos o **pipeline** como argumento do gridsearch!

Para maiores informações, [clique aqui](https://scikit-learn.org/stable/modules/grid_search.html#grid-search). 

Vamos ver a seguir como esta estrutura funciona:

In [10]:
from sklearn.model_selection import GridSearchCV

param_grid = {'knn__n_neighbors': np.arange(1, 20),
              'knn__weights':['uniform', 'distance'], 
              'knn__metric':["euclidean", 'manhattan', "chebyshev"]}

# os quatro principais principais parâmetros!
grid = GridSearchCV(pipe, param_grid, scoring="f1_weighted", cv=5)

# treine o objeto da grade com os dados de treino]
grid.fit(X_train, y_train)

# faça previsões (COM O MELHOR MODELO) a partir as features de teste
y_pred = grid.predict(X_test)

# avalie o MELHOR MODELO
from sklearn.metrics import classification_report, confusion_matrix

print("Matriz de confusão do modelo nos dados de teste:\n")
print(confusion_matrix(y_test, y_pred))

print("\nMatriz de confusão do modelo nos dados de teste:\n")
print(classification_report(y_test, y_pred))

Matriz de confusão do modelo nos dados de teste:

[[39  3]
 [ 2 70]]

Matriz de confusão do modelo nos dados de teste:

              precision    recall  f1-score   support

           0       0.95      0.93      0.94        42
           1       0.96      0.97      0.97        72

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



O Gridsearch avalia o melhor modelo segundo a métrica que passamos em `scoring`. [Clique aqui](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter) para ver as métricas disponíveis por padrão.

### 3.1) **Cross validation** <a class="anchor" id="crossvalidation"></a>

Mas o GS vai além: ele não calcula a métrica uma única vez, mas sim **várias vezes**, conforme especificado pelo parâmetro `cv`. No fim, o melhor modelo é o que tem o melhor scoring **médio** entre as vezes que é calculado.

O "cv" quer dizer **Cross Validation**, o método mais seguro de realizar um treinamento e avaliação de um modelo:

<center><img src="https://ethen8181.github.io/machine-learning/model_selection/img/kfolds.png" width=600></center>

Naturalmente, cada um dos folds são feitos de modo **aleatório**, garantindo assim uma avaliação justa do modelo, **e que faça uso de toda a base de dados!!**

O GS treinado tem diversos atributos super úteis e interessantes! Vamos dar uma olhada neles:

`.best_estimator_`: retorna quais as escolhas do Pipeline que produziram o melhor modelo. No que diz respeito ao modelo, temos exatamente quais os parâmetros escolhidos!

In [11]:
grid.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('knn',
                 KNeighborsClassifier(metric='manhattan', n_neighbors=6))])

`.best_params_`: retorna os parâmetros testados na grade que produziram o melhor modelo.

In [12]:
grid.best_params_

{'knn__metric': 'manhattan', 'knn__n_neighbors': 6, 'knn__weights': 'uniform'}

`.best_score_`: retorna a **média cross-validada da métrica de interesse** do melhor modelo. Como esse é o valor médio construído usando CV, este é estatisticamente o valor mais realístico a ser atribuído à performance do modelo!

In [13]:
grid.best_score_

0.9734787920524994

`.cv_results_`: diversas informações do processo feito pelo GS

In [14]:
grid.cv_results_

{'mean_fit_time': array([0.00553598, 0.00273795, 0.00275807, 0.00270629, 0.00271964,
        0.00281868, 0.00271397, 0.00267792, 0.00267673, 0.00279961,
        0.00266132, 0.00261197, 0.00264082, 0.00267153, 0.00268278,
        0.00262785, 0.00268602, 0.00269227, 0.00278387, 0.00267005,
        0.00260015, 0.00267439, 0.00264459, 0.00267844, 0.00280418,
        0.00266933, 0.00267472, 0.00262294, 0.00280728, 0.00264564,
        0.00267873, 0.00265026, 0.00262814, 0.00282407, 0.00274854,
        0.00266037, 0.00277495, 0.0026577 , 0.00275393, 0.00289717,
        0.00259991, 0.00261636, 0.00262499, 0.00303693, 0.00307245,
        0.00258174, 0.00260463, 0.00266385, 0.00333395, 0.00265274,
        0.00266576, 0.00264072, 0.00259538, 0.00273781, 0.002671  ,
        0.00263243, 0.00266552, 0.00267291, 0.00263767, 0.00265675,
        0.00262594, 0.00260558, 0.00264101, 0.00254655, 0.00277534,
        0.00266285, 0.00265102, 0.00264926, 0.00260425, 0.00259933,
        0.00263567, 0.00260668,